In [77]:
import pandas as pd
import umap.umap_ as umap
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import FastICA 

path_file = "data/pathwayscores_clinicalinformation_METABRIC.tsv"
data = pd.read_csv(path_file, sep='\t')

In [11]:
features = ['Cell_Cycle', 'HIPPO', 'MYC', 'NOTCH', 'NRF2', 'PI3K', 'TGF.Beta', 'RTK_RAS', 'TP53', 'WNT', 'Hypoxia', 'SRC', 'ESR1', 'ERBB2']
target = ['PAM50']

# Separating out the features
x = data.loc[:, features].values

# Separating out the target
y = data.loc[:,target].values

In [78]:
dimension=2

def reduce_dimension(method, dimension, x, kernel=None):
    if kernel == None:
        kernel = ""
    if method == "pca":
        pca = PCA(n_components=dimension)
        X_reduced = pd.DataFrame(pca.fit_transform(x))

    elif method == "kpca":
        transformer = KernelPCA(n_components=7, kernel=kernel)
        X_reduced = pd.DataFrame(transformer.fit_transform(x))

    elif method == "tsne":
        X_reduced = pd.DataFrame(TSNE(n_components=dimension, random_state=6).fit_transform(x))
    
    # Factor Analysis
    elif method == "fa":
        X_reduced = pd.DataFrame(FactorAnalysis(n_components = dimension).fit_transform(x))
    # Independent Component Analysis
    elif method == "ica":
        X_reduced = pd.DataFrame(FastICA(n_components=dimension, random_state=12).fit_transform(x))

    elif method == "isomap":
        X_reduced = pd.DataFrame(Isomap(n_neighbors=5, n_components=dimension).fit_transform(x))

    elif method == "umap":
        X_reduced = pd.DataFrame(umap.UMAP(n_neighbors=5, min_dist=0.3, n_components=dimension).fit_transform(x))
    finalDf = pd.concat([data.loc[:,target], X_reduced], axis = 1)

    fig = px.scatter(x = finalDf.iloc[:,1], y = finalDf.iloc[:,2], hover_data=[finalDf.iloc[:,0]], color = finalDf.iloc[:,0], marginal_x = 'box', marginal_y = 'box', title = method + " " + kernel)

    return fig.show()

In [80]:
# 'pca', 'kpca', 'tsne', 'fa', 'ica', 'isomap', 'umap'
method = "umap"
# ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘cosine’
kernel = ""

reduce_dimension(method, dimension, x, kernel)